# Импорт библиотек и подгрузка данных

In [51]:
import pandas as pd
import numpy as np


# читаю csv-файл в переменную
diabetes = pd.read_csv('data/diabetes_data.csv', sep=',')
# создаю копию оригинального датафрейма, для безопасной обработки данных
processing_diabetes = diabetes.copy()

# Задание 8.1 - 8.9

    Pregnancies — количество беременностей.
    Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.
    BloodPressure — диастолическое артериальное давление (мм рт. ст.).
    SkinThickness — толщина кожной складки трицепса (мм).
    Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).
    BMI — индекс массы тела (
    DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).
    Age — возраст.
    Outcome — наличие диабета (0 — нет, 1 — да).

In [2]:
display(diabetes.head())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


## Задание 8.1

Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [ ]:
# смотрим на количество строк и столбцов
diabetes.shape

(778, 10)

In [52]:
# создаем маску для того чтобы узнать сколько есть дубликатов строк 
mask = processing_diabetes.duplicated()
# осталяем только дубликаты
# в качестве маски идет раяд с булевыми переменными 
diabetes_dupl = processing_diabetes[mask]
# смотрим на количество дубликатов
display(diabetes_dupl.shape[0])
# удаляем эти 10 строк
processing_diabetes = processing_diabetes.drop_duplicates()
# проверяем, что все десять строк удалились
display(processing_diabetes.shape[0])


10

768

## Задние 8.2

Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [53]:
# создаю пустой сиписок, который будет поплянться неинформативыными признаками 
uninformative_list = list()
# создаем цикл, чтобы рассмотреть каждый признак 
for col in processing_diabetes.columns:
    # находим максимальную повторяемость
    top_freq = processing_diabetes[col].value_counts(normalize=True).max()
    # нахожу уникальные записи и перевожу все в процентное соотношение,
    # чтобы инициализировать условия
    unique_ratio = processing_diabetes[col].nunique() / processing_diabetes[col].count()
    # проверяю условие на прохождение границы порогов в 95 процентов
    if top_freq > 0.95:
        # пополняю список именами признаков для удаления их из исходной таблыицы
        uninformative_list.append(col)
    if unique_ratio > 0.95:
        uninformative_list.append(col)

# удаляю неинформативные признаки
processing_diabetes = processing_diabetes.drop(uninformative_list, axis=1)
# утверждаю удаление дубликатов и неинформативных признаков
display(processing_diabetes)
processing_diabetes.shape



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...,...
763,5,139,64,35,140,28.6,0.411,26,0
764,1,96,122,0,0,22.4,0.207,27,0
765,10,101,86,37,0,45.6,1.136,38,1
766,0,141,0,0,0,42.4,0.205,29,1


(768, 9)

## Задание 8.3

Попробуйте найти пропуски в данных с помощью метода isnull().

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [54]:
# подтверждаем, что при помощи метода isnull пропусков знчений нет
processing_diabetes.isnull()
# создаю фнкцию которая поменяет записи со значением ноль на Nan
def zero_change(value):
    # проверяю что значение 0
    if value == 0:
        # переназначаю на np.nan
        value = np.nan
        return value
    # иначе ничего не меняем
    else:
        return value

# пробегаюсь по каждому признаку и применяю функцию zero_change 
# для замены 0 на nan
for col in processing_diabetes.columns:
    # применяю функцию для улучшения данных
    # обязательно переопределяю признак
    processing_diabetes[col] = processing_diabetes[col].apply(zero_change)
      
# узнаю какая доля пропусков в стодбце Insulin
display(processing_diabetes['Insulin'].isnull().value_counts(normalize=True))
# округляю до двух знаков после запятой
insulin_precent = processing_diabetes['Insulin'].isnull()\
    .value_counts(normalize=True)
round(insulin_precent, 2)
    

Insulin
False    0.513021
True     0.486979
Name: proportion, dtype: float64

Insulin
False    0.51
True     0.49
Name: proportion, dtype: float64

## Задание 8.4

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [55]:
# узнаю число пропусков данных в процентном соотношении
diabetes_percent = processing_diabetes.isnull().mean() * 100
# фильтрую и оствляю столбцы только с пропусками, сортирую для наглядности
diabetes_with_null = diabetes_percent[diabetes_percent > 0].sort_values(\
    ascending=False)
display(diabetes_percent)
# утверждаем, какие признаки подлежат к удалению
display(round(diabetes_with_null))
# создаю маску для того чтобы направить ее в качестве аргумента
# и порога, прохождение  которого, потребует от метода  dropna удаления
thresh = processing_diabetes.shape[0] * 0.7
# удаляем признаки где число пропусков сотавляет более 30 процентов
dropna_diabetes = processing_diabetes.dropna(thresh=thresh, axis=1)
display(processing_diabetes)
display(dropna_diabetes)


Pregnancies                 14.453125
Glucose                      0.651042
BloodPressure                4.557292
SkinThickness               29.557292
Insulin                     48.697917
BMI                          1.432292
DiabetesPedigreeFunction     0.000000
Age                          0.000000
Outcome                     65.104167
dtype: float64

Outcome          65.0
Insulin          49.0
SkinThickness    30.0
Pregnancies      14.0
BloodPressure     5.0
BMI               1.0
Glucose           1.0
dtype: float64

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,98.0,58.0,33.0,190.0,34.0,0.430,43,NaN
1,2.0,112.0,75.0,32.0,NaN,35.7,0.148,21,NaN
2,2.0,108.0,64.0,NaN,NaN,30.8,0.158,21,NaN
3,8.0,107.0,80.0,NaN,NaN,24.6,0.856,34,NaN
4,7.0,136.0,90.0,NaN,NaN,29.9,0.210,50,NaN
...,...,...,...,...,...,...,...,...,...
763,5.0,139.0,64.0,35.0,140.0,28.6,0.411,26,NaN
764,1.0,96.0,122.0,NaN,NaN,22.4,0.207,27,NaN
765,10.0,101.0,86.0,37.0,NaN,45.6,1.136,38,1.0
766,NaN,141.0,NaN,NaN,NaN,42.4,0.205,29,1.0


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age
0,6.0,98.0,58.0,33.0,34.0,0.430,43
1,2.0,112.0,75.0,32.0,35.7,0.148,21
2,2.0,108.0,64.0,NaN,30.8,0.158,21
3,8.0,107.0,80.0,NaN,24.6,0.856,34
4,7.0,136.0,90.0,NaN,29.9,0.210,50
...,...,...,...,...,...,...,...
763,5.0,139.0,64.0,35.0,28.6,0.411,26
764,1.0,96.0,122.0,NaN,22.4,0.207,27
765,10.0,101.0,86.0,37.0,45.6,1.136,38
766,NaN,141.0,NaN,NaN,42.4,0.205,29


## Задание 8.5

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [58]:
# создаю минимальный порог непустых значений 
thresh = dropna_diabetes.shape[1] - 2
# удаляю строки в которых количество пропусков более двух
processing_diabetes = dropna_diabetes.dropna(thresh=thresh, axis=0)
display(processing_diabetes.shape)
print(f'Результырующее число записей: {processing_diabetes.shape[0]}')

(752, 7)

Результырующее число записей: 752


## Задание 8.6

В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых. 

In [61]:
diabetes_percent = processing_diabetes.isnull().mean() * 100
diabetes_with_null = diabetes_percent[diabetes_percent > 0]
diabetes_percent
diabetes_with_null


Pregnancies      13.430851
Glucose           0.664894
BloodPressure     2.526596
SkinThickness    28.058511
BMI               0.531915
dtype: float64

In [67]:
# создаю словарь в котором буду содержать значения для замены NaN
# везде заменяю на медиану
values = {
    'Pregnancies': processing_diabetes['Pregnancies'].median(),
    'Glucose': processing_diabetes['Glucose'].median(),
    'BloodPressure': processing_diabetes['BloodPressure'].median(),
    'SkinThickness': processing_diabetes['SkinThickness'].median(),
    'BMI': processing_diabetes['BMI'].median()
}
# заменяем значения при помощи метода fillna, и готового словаря values
processing_diabetes = processing_diabetes.fillna(values)
# утверждаю, что пропксков больше нет
display(processing_diabetes.isnull().mean())

Pregnancies                 0.0
Glucose                     0.0
BloodPressure               0.0
SkinThickness               0.0
BMI                         0.0
DiabetesPedigreeFunction    0.0
Age                         0.0
dtype: float64

## Задание 8.7

Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [73]:
display(processing_diabetes)
# создаю функцию для нахождения выбросов
def tyuki_funk(data, feature, log_scale=False, left=1.5, right=1.5):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]

    return outliers, cleaned

# помещаю датафрейм из выбросов и датафрейм очищенный от выбросов в переменную
outliers, cleaned = tyuki_funk(processing_diabetes, 'SkinThickness')
# смотрим на то сколько нашдось выбросов в методе межквартильного размаха
print(f'Результирующее количество выбросов: {outliers.shape[0]}')

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age
0,6.0,98.0,58.0,33.0,34.0,0.430,43
1,2.0,112.0,75.0,32.0,35.7,0.148,21
2,2.0,108.0,64.0,29.0,30.8,0.158,21
3,8.0,107.0,80.0,29.0,24.6,0.856,34
4,7.0,136.0,90.0,29.0,29.9,0.210,50
...,...,...,...,...,...,...,...
762,2.0,122.0,76.0,27.0,35.9,0.483,26
763,5.0,139.0,64.0,35.0,28.6,0.411,26
764,1.0,96.0,122.0,29.0,22.4,0.207,27
765,10.0,101.0,86.0,37.0,45.6,1.136,38


Результирующее количество выбросов: 81


## Задание 8.8

Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [75]:
# определяем функцию для нахождения выбросов по методу z-отклонений
def z_funk(data, feature, log_scale=False, left=3, right=3):
    x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    
    return outliers, cleaned

# помещаю в переменные таблицу только с выбросами и таблицу без выбсосов
outliers, cleaned = z_funk(processing_diabetes, 'SkinThickness')
# печатаем вывод для наглядности
print(f'Результирующее количество выбросов: {outliers.shape[0]}')
    

Результирующее количество выбросов: 4


## Задание 8.9

На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.



Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

In [77]:
# создаю функцию для нахождения выбросов
def tyuki_funk(data, feature, log_scale=False, left=1.5, right=1.5):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]

    return outliers, cleaned

# для нахождения разницы в количестве выбросов в выводе без логорифмирования
# и при логорифмировании создаем разные переменные и вкладываем туда 
# функцию при log_scale=False и при log_scale=True
outliers, cleaned = tyuki_funk(processing_diabetes, 'DiabetesPedigreeFunction')
outliers_log, cleaned_log = tyuki_funk(processing_diabetes, \
    'DiabetesPedigreeFunction', log_scale=True)
# печатаем вывод для лучшего понимания
print(f'Разница между вывдами в логорифмическом масштабе и без составляет:\
    {outliers.shape[0] - outliers_log.shape[0]}')

Разница между вывдами в логорифмическом масштабе и без составляет:    29
